In [159]:
import pandas as pd
import janitor

In [160]:
# Group and keep columns with 'as_index'
# https://stackoverflow.com/questions/31569549/how-to-groupby-a-dataframe-in-pandas-and-keep-columns

### Daily report data

In [161]:
data = pd.read_csv('filter.csv', encoding='utf-8')

In [162]:
print(data.set_index(['ID_REGISTRO']).index.is_unique) # ids are not unique... it's unclear how reports are aggregated

False


In [163]:
data['FECHA_INGRESO'] = pd.to_datetime(data.FECHA_INGRESO, format='%Y-%m-%d', errors='coerce')
data['FECHA_INGRESO'] = data['FECHA_INGRESO'].dt.strftime('%Y-%m-%d')
data = data.set_index(pd.DatetimeIndex(data['FECHA_INGRESO'])).sort_index()

In [164]:
len(data)

163973

### Note
Daily report for april 20 (ie, reported in 21 april) stops using testing site (ENTIDAD_UM),
<br>
And starts using region of residence (ENTIDAD_RES) so we shall treat both sets of data accordingly

In [165]:
dat = data.copy()

In [166]:
tbl_x = dat.filter_date('FECHA_INGRESO','2020-01-06','2020-04-20')
tbl_x = tbl_x.drop(['ENTIDAD_RES'], axis=1) # drop by region
tbl_x = tbl_x.set_index(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_UM', 'ID_REGISTRO', 'PAIS_ORIGEN']
)
tbl_x = tbl_x.groupby(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_UM', 'ID_REGISTRO', 'PAIS_ORIGEN'], as_index=True).count().reset_index()
tbl_x = tbl_x.rename(columns={'ENTIDAD_UM':'Region_ID'})

In [167]:
tbl_y = dat.filter_date('FECHA_INGRESO','2020-04-20','2022-12-31') # the long con
tbl_y = tbl_y.drop(['ENTIDAD_UM'], axis=1) # drop by region
tbl_y = tbl_y.set_index(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_RES', 'ID_REGISTRO', 'PAIS_ORIGEN']
)
tbl_y = tbl_y.groupby(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_RES', 'ID_REGISTRO', 'PAIS_ORIGEN'], as_index=True).count().reset_index()
tbl_y = tbl_y.rename(columns={'ENTIDAD_RES':'Region_ID'})

In [168]:
tbl = pd.concat([tbl_x, tbl_y])

In [169]:
tbl

,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,Region_ID,ID_REGISTRO,PAIS_ORIGEN,RESULTADO
0,2020-01-06,2020-01-06,9999-99-99,18,09f6d9,Local,9
1,2020-01-08,2020-01-08,9999-99-99,30,1c29f8,Local,8
2,2020-01-14,2020-01-14,9999-99-99,1,0ceb0f,Local,1
3,2020-01-27,2020-01-27,9999-99-99,15,1879e0,Local,1
4,2020-02-07,2020-02-07,9999-99-99,11,03e780,Local,6
...,...,...,...,...,...,...,...
5087,2020-04-27,2020-04-26,9999-99-99,29,06a5f6,Local,1
5088,2020-04-27,2020-04-27,9999-99-99,28,0ce4e9,Local,1
5089,2020-04-27,2020-04-27,9999-99-99,29,052d4e,Local,1
5090,2020-04-27,2020-04-27,9999-99-99,29,15c086,Local,1


In [170]:
tbl = tbl.rename(
    columns={
        'FECHA_INGRESO':'Date_Confirmed',
        'FECHA_SINTOMAS':'Date_Symptoms',
        'ID_REGISTRO':'Case_ID',
        'RESULTADO':'Status',
        'PAIS_ORIGEN':'Origin'
    })

In [171]:
tbl['Status'] = '1'

In [172]:
tbl

,Date_Confirmed,Date_Symptoms,FECHA_DEF,Region_ID,Case_ID,Origin,Status
0,2020-01-06,2020-01-06,9999-99-99,18,09f6d9,Local,1
1,2020-01-08,2020-01-08,9999-99-99,30,1c29f8,Local,1
2,2020-01-14,2020-01-14,9999-99-99,1,0ceb0f,Local,1
3,2020-01-27,2020-01-27,9999-99-99,15,1879e0,Local,1
4,2020-02-07,2020-02-07,9999-99-99,11,03e780,Local,1
...,...,...,...,...,...,...,...
5087,2020-04-27,2020-04-26,9999-99-99,29,06a5f6,Local,1
5088,2020-04-27,2020-04-27,9999-99-99,28,0ce4e9,Local,1
5089,2020-04-27,2020-04-27,9999-99-99,29,052d4e,Local,1
5090,2020-04-27,2020-04-27,9999-99-99,29,15c086,Local,1


In [173]:
#tbl[~tbl.FECHA_DEF.str.startswith(('9999'))]
tbl = tbl[tbl.FECHA_DEF.str.startswith(('9999'))] # Not deceased

In [174]:
tbl = tbl.drop(['FECHA_DEF'], axis=1)

In [175]:
tbl

,Date_Confirmed,Date_Symptoms,Region_ID,Case_ID,Origin,Status
0,2020-01-06,2020-01-06,18,09f6d9,Local,1
1,2020-01-08,2020-01-08,30,1c29f8,Local,1
2,2020-01-14,2020-01-14,1,0ceb0f,Local,1
3,2020-01-27,2020-01-27,15,1879e0,Local,1
4,2020-02-07,2020-02-07,11,03e780,Local,1
...,...,...,...,...,...,...
5087,2020-04-27,2020-04-26,29,06a5f6,Local,1
5088,2020-04-27,2020-04-27,28,0ce4e9,Local,1
5089,2020-04-27,2020-04-27,29,052d4e,Local,1
5090,2020-04-27,2020-04-27,29,15c086,Local,1


In [176]:
#tbl.to_csv('confirmed_by_date.csv', index=False)

In [177]:
geo = pd.read_csv('../../data/dge/meta_data/entidades.csv')

In [178]:
geo.keys()

Index(['CLAVE_ENTIDAD', 'ENTIDAD_FEDERATIVA', 'ABREVIATURA'], dtype='object')

In [179]:
geo = geo.rename(columns={'CLAVE_ENTIDAD':'Region_ID', 'ENTIDAD_FEDERATIVA':'Region', 'ABREVIATURA':'Region_Key'})

In [180]:
output = pd.merge(tbl, geo, on='Region_ID') 

In [181]:
output = output[['Date_Confirmed', 'Date_Symptoms', 'Region', 'Region_ID', 'Region_Key', 'Origin']]

In [182]:
output

,Date_Confirmed,Date_Symptoms,Region,Region_ID,Region_Key,Origin
0,2020-01-06,2020-01-06,NAYARIT,18,NT,Local
1,2020-03-18,2020-03-17,NAYARIT,18,NT,Local
2,2020-03-19,2020-03-10,NAYARIT,18,NT,Local
3,2020-03-23,2020-03-19,NAYARIT,18,NT,Local
4,2020-03-24,2020-03-21,NAYARIT,18,NT,Local
...,...,...,...,...,...,...
17367,2020-04-26,2020-04-26,TLAXCALA,29,TL,Local
17368,2020-04-26,2020-04-26,TLAXCALA,29,TL,Local
17369,2020-04-27,2020-04-26,TLAXCALA,29,TL,Local
17370,2020-04-27,2020-04-27,TLAXCALA,29,TL,Local


In [183]:
output['Region'] = output['Region'].str.replace('COAHUILA DE ZARAGOZA','COAHUILA')
output['Region'] = output['Region'].str.replace('MICHOACÁN DE OCAMPO','MICHOACAN')
output['Region'] = output['Region'].str.replace('VERACRUZ DE IGNACIO DE LA LLAVE','VERACRUZ')

In [184]:
output.Region.unique()

array(['NAYARIT', 'VERACRUZ', 'AGUASCALIENTES', 'MÉXICO', 'GUANAJUATO',
       'BAJA CALIFORNIA', 'CIUDAD DE MÉXICO', 'COAHUILA', 'SINALOA',
       'CHIAPAS', 'YUCATÁN', 'MORELOS', 'NUEVO LEÓN', 'QUERÉTARO',
       'PUEBLA', 'DURANGO', 'JALISCO', 'QUINTANA ROO', 'CAMPECHE',
       'SAN LUIS POTOSÍ', 'GUERRERO', 'OAXACA', 'COLIMA', 'CHIHUAHUA',
       'SONORA', 'TAMAULIPAS', 'HIDALGO', 'TABASCO', 'MICHOACAN',
       'BAJA CALIFORNIA SUR', 'ZACATECAS', 'TLAXCALA'], dtype=object)

In [185]:
output.Origin.unique()

array(['Local', 'Estados Unidos de AmÃ©rica', 'Estados Unidos de América',
       'Venezuela', 'PerÃº', 'Perú', 'El Salvador', 'Brasil', 'Cuba',
       'Guatemala', 'Canadá', 'Alemania', 'RepÃºblica de Honduras',
       'República de Honduras', 'Otro', 'CamerÃºn', 'Camerún', 'Chile'],
      dtype=object)

In [186]:
output.to_csv('../../latest.csv', index=False)